Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p0">Part 0</a>: Kaggle Competition
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

*Note:* Each time we call the pipeline during grid search, each component is fit again. The vectorizer (tf-idf) is transforming our entire vocabulary during each cross-validation fold. That transformation adds significant run time to our grid search. There *might* be interactions between the vectorizer and our classifier, so we estimate their performance together in the code below. However, if your goal is to reduce run time. Train your vectorizer separately (ie out of the grid-searched pipeline). 

In [3]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [8]:
#print out data sample
data['data'][0]

'From: mangoe@cs.umd.edu (Charley Wingate)\nSubject: Benediktine Metaphysics\nLines: 24\n\nBenedikt Rosenau writes, with great authority:\n\n>     IF IT IS CONTRADICTORY IT CANNOT EXIST.\n\n"Contradictory" is a property of language.  If I correct this to\n\n\n      THINGS DEFINED BY CONTRADICTORY LANGUAGE DO NOT EXIST\n\nI will object to definitions as reality.  If you then amend it to\n\n      THINGS DESCRIBED BY CONTRADICTORY LANGUAGE DO NOT EXIST\n\nthen we\'ve come to something which is plainly false.  Failures in\ndescription are merely failures in description.\n\n(I\'m not an objectivist, remember.)\n\n\n-- \nC. Wingate        + "The peace of God, it is no peace,\n                  +    but strife closed in the sod.\nmangoe@cs.umd.edu +  Yet, brothers, pray for but one thing:\ntove!mangoe       +    the marv\'lous peace of God."\n'

In [9]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [10]:
# Define the Pipeline
pipe = Pipeline([
                 # Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', rfc)
                ])

# The pipeline puts together a bunch fit then transform,fit then predict. 

In [11]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   50.4s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  2.8min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [12]:
grid_search.best_score_

0.8833138856476079

In [13]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([1, 1])

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [4]:
import pandas as pd
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

### Define Pipeline Components

In [18]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier(n_estimators=100)

pipe = Pipeline([('vect', vect), ('clf', clf)])

In [5]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [28]:
parameters = {
    'vect__max_df': uniform(0.75, 1.0),
    'clf__max_depth': uniform(5, 20)
}

random_search = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=parameters,
    cv=5,
    n_jobs=-1,
    verbose=1
)
random_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.6min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'vect__max_df': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1186e3048>, 'clf__max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x11ca95be0>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=1)

In [29]:
print(random_search.best_score_, random_search.best_params_)

0.7066307805236115 {'clf__max_depth': 24.54970664877203, 'vect__max_df': 1.421945489687983}


### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this compeition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [20]:
# Predictions on test sample
pred = random_search.predict(test['description'])

In [21]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [22]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [23]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission1.csv', index=False)

## Challenge

You're trying to achieve 90% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

In [6]:
from scipy.stats import randint

In [7]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100,  # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

In [47]:
params = { 
    'lsi__svd__n_components': randint(10, 250),
    'lsi__vect__max_df': uniform(0.9, 1.0),
    'clf__n_estimators': randint(5, 20)
}

In [48]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', clf)])

In [49]:
print(pipe)

Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])


In [50]:
# Fit
random_search = RandomizedSearchCV(
    pipe, params, cv=5, n_jobs=-1, verbose=10, random_state=42
)
random_search.fit(data.data, data.target)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  7.0min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'lsi__svd__n_components': <scipy.stats._distn_infrastructure.rv_frozen object at 0x11bff9358>, 'lsi__vect__max_df': <scipy.stats._distn_infrastructure.rv_frozen object at 0x11bff9208>, 'clf__n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x11b485f60>},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbos

In [51]:
random_search.best_score_

0.9054842473745625

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [52]:
lsi = Pipeline([('vect', vect), ('svd', svd)])
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier(n_estimators=100)

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [54]:
params = { 
    'lsi__svd__n_components': randint(10, 250),
    'lsi__vect__max_df': uniform(0.7, 1.0),
    'clf__n_estimators': randint(5, 20)
}

random_search = RandomizedSearchCV(
    pipe, params, cv=5, n_jobs=-1, verbose=10, random_state=42)
random_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  8.9min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'lsi__svd__n_components': <scipy.stats._distn_infrastructure.rv_frozen object at 0x11c7b8668>, 'lsi__vect__max_df': <scipy.stats._distn_infrastructure.rv_frozen object at 0x11c784940>, 'clf__n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x11c784c18>},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbos

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [55]:
# Predictions on test sample
pred = random_search.predict(test['description'])

In [56]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [57]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [58]:
subNumber = 2

In [59]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [8]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [61]:
doc = nlp("Two bananas in pyjamas")

In [62]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [9]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [64]:
X = get_word_vectors(data.data)

len(X) == len(data.data)

True

In [65]:
rfc.fit(X, data.target)

/Users/weilu/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [66]:
rfc.score(X, data.target)

0.9801633605600933

## Follow Along

In [10]:
# Apply to your Dataset
X = get_word_vectors(train['description'])

len(X) == len(train['description'])

True

In [69]:
from sklearn.ensemble import GradientBoostingClassifier

param_dist = {
    'max_depth': randint(3, 6),
    'min_samples_leaf': randint(3, 6),
    'max_features': ['auto', 'sqrt', 'log2']
}

In [71]:
clf = GradientBoostingClassifier()

search = RandomizedSearchCV(
    clf, param_dist, cv=5, n_jobs=-1, n_iter=15, verbose=10, random_state=42
)
search.fit(X, train['ratingCategory'])

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 10.0min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 11.9min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 13.5min
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed: 20.2min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=15, n_jobs=-1,
          param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x129d9fcc0>, 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x129d9fc88>, 'max_features': ['auto', 'sqrt', 'log2']},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=10)

In [77]:
search.best_score_

0.7421091264986542

In [78]:
# Predictions on test sample
pred = search.predict(get_word_vectors(test['description']))

In [79]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [80]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [81]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

In [11]:
from sklearn.svm import LinearSVC
from numpy import logspace, linspace

svd = TruncatedSVD(algorithm='randomized',
                   n_iter=10)
vect = TfidfVectorizer(ngram_range=(1,3))
lsi = Pipeline([('vect', vect), ('svd', svd)])

clf = LinearSVC(dual=False)

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

In [29]:
from sklearn.naive_bayes import GaussianNB
from numpy import logspace, linspace

svd = TruncatedSVD(algorithm='randomized',
                   n_iter=10)
vect = TfidfVectorizer(ngram_range=(1,3))
lsi = Pipeline([('vect', vect), ('svd', svd)])

clf = SGDClassifier(tol=1e-3)

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

In [30]:
tokenizer = vect.build_tokenizer()
stop_words = vect.get_stop_words()

In [32]:
param_dist = {
    'lsi__svd__n_components': range(10, 260, 10),
    'lsi__vect__max_df': linspace(0.9, 0.95),
    'lsi__vect__min_df': randint(3, 10),
    'lsi__vect__stop_words': (None, 'english', stop_words),
    'lsi__vect__tokenizer': (None, tokenizer),
    'clf__max_iter': range(10, 110, 10)
}

search = RandomizedSearchCV(
    pipe, param_dist, cv=5, n_iter=30, n_jobs=-1, verbose=10, scoring='f1_micro', random_state=42, return_train_score=False
)
search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   23.7s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   30.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   45.3s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   52.1s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  5.6min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm=..._state=None, shuffle=True, tol=0.001,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
          fit_params=None, iid='warn', n_iter=30, n_jobs=-1,
          param_distributions={'lsi__svd__n_components': range(10, 260, 10), 'lsi__vect__max_df': array([0.9    , 0.90102, 0.90204, 0.90306, 0.90408, 0.9051 , 0.90612,
       0.90714, 0.90816, 0.90918, 0.9102 , 0.91122, 0.91224, 0.91327,
       0.91429, 0.91531, 0.91633, 0.91735, 0.91837, 0.91939, 0.92041,
  ...orizerMixin.build_tokenizer.<locals>.<lambda> at 0x1402596a8>), 'clf__max_iter': range(10, 110, 10)},
          pre

In [33]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,5,13,16,27,17,2,11,3,15,0,...,1,24,26,21,18,28,20,4,22,23
mean_fit_time,11.4235,10.6755,24.8245,7.75531,16.2606,7.64584,8.49806,6.89832,25.6396,10.2184,...,6.04601,4.13701,5.30427,3.913,5.7999,3.44914,4.2207,3.23621,4.26974,4.46506
std_fit_time,0.61981,0.395583,0.734173,0.257789,1.6485,0.621501,0.3671,0.228881,0.454923,0.698836,...,0.459549,0.066929,0.156892,0.256438,0.772984,0.128315,0.1241,0.0789467,0.184812,0.0666402
mean_score_time,0.494873,0.490443,0.563446,0.441899,0.587846,0.324707,0.375408,0.310829,0.649867,0.39903,...,0.422745,0.304236,0.417689,0.303435,0.47457,0.30084,0.37961,0.332388,0.387268,0.407373
std_score_time,0.0802542,0.0561794,0.0932081,0.0273514,0.100665,0.0258157,0.0223187,0.042052,0.127039,0.018469,...,0.0162367,0.0217469,0.0273079,0.0234958,0.0929191,0.00835811,0.0157814,0.0616387,0.0109245,0.0188824
param_clf__max_iter,90,80,10,40,80,80,60,20,50,70,...,50,70,40,80,30,60,20,50,30,30
param_lsi__svd__n_components,170,140,250,130,240,210,220,240,240,200,...,70,90,50,50,30,20,10,10,10,10
param_lsi__vect__max_df,0.926531,0.922449,0.906122,0.931633,0.910204,0.935714,0.941837,0.943878,0.92551,0.928571,...,0.918367,0.914286,0.922449,0.934694,0.932653,0.941837,0.947959,0.911224,0.934694,0.904082
param_lsi__vect__min_df,5,6,3,9,5,5,6,8,3,9,...,9,9,9,6,5,7,9,4,7,4
param_lsi__vect__stop_words,None,None,None,None,None,english,english,english,None,None,...,None,english,None,english,None,english,None,english,None,None
param_lsi__vect__tokenizer,<function VectorizerMixin.build_tokenizer.<loc...,<function VectorizerMixin.build_tokenizer.<loc...,<function VectorizerMixin.build_tokenizer.<loc...,None,<function VectorizerMixin.build_tokenizer.<loc...,None,None,<function VectorizerMixin.build_tokenizer.<loc...,None,<function VectorizerMixin.build_tokenizer.<loc...,...,None,<function VectorizerMixin.build_tokenizer.<loc...,None,None,<function VectorizerMixin.build_tokenizer.<loc...,None,<function VectorizerMixin.build_tokenizer.<loc...,None,<function VectorizerMixin.build_tokenizer.<loc...,None


In [140]:
param_dist = {
    'C': logspace(-1, 1.5, 100),
}

search = RandomizedSearchCV(
    clf, param_dist, cv=5, n_jobs=-1, n_iter=30, verbose=10, random_state=42, scoring='f1_micro', return_train_score=False
)
search.fit(X, train['ratingCategory'])

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   30.2s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   34.6s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   41.4s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:   50.5s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:   59.5s
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.6min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          fit_params=None, iid='warn', n_iter=30, n_jobs=-1,
          param_distributions={'C': array([ 0.1    ,  0.10599, ..., 29.83647, 31.62278])},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score=False, scoring='f1_micro', verbose=10)

In [141]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,19,11,13,22,26,5,4,23,10,3,...,2,29,17,7,12,16,0,21,14,27
mean_fit_time,1.45776,1.45056,1.73582,1.62135,2.01624,2.06875,2.11187,2.19605,1.13989,2.2089,...,2.94495,2.2558,2.8941,2.68906,2.62449,2.64208,3.02715,3.3647,3.28578,3.18151
std_fit_time,0.0352315,0.0598483,0.0887616,0.044305,0.208596,0.139674,0.0747418,0.131728,0.143721,0.0563313,...,0.198165,0.410511,0.0771506,0.155138,0.176611,0.238319,0.245176,0.214827,0.121201,0.0830972
mean_score_time,0.0243655,0.0140067,0.0157608,0.0118012,0.010419,0.0176309,0.012545,0.0114984,0.0105288,0.0132841,...,0.0115625,0.00971684,0.0137752,0.0114573,0.00852723,0.0121338,0.0179466,0.0126765,0.0202052,0.0153443
std_score_time,0.00787853,0.00559935,0.00749222,0.00328671,0.00120522,0.00598955,0.00347132,0.00366732,0.00312749,0.0034643,...,0.00183912,0.00335673,0.00421025,0.00423994,0.00132956,0.00334467,0.00505765,0.00438632,0.0109569,0.00551452
param_C,0.606496,0.572237,0.681292,0.453488,1.02353,0.965714,1.29155,1.14976,0.284804,1.36887,...,5.85702,6.57933,8.79923,10.4762,6.97324,8.30218,12.4727,16.681,18.7382,26.5609
params,{'C': 0.60649647746946},{'C': 0.5722367659350217},{'C': 0.6812920690579614},{'C': 0.4534878508128582},{'C': 1.0235310218990263},{'C': 0.9657139052966048},{'C': 1.291549665014884},{'C': 1.1497569953977356},{'C': 0.2848035868435802},{'C': 1.3688745095370807},...,{'C': 5.857020818056667},{'C': 6.579332246575679},{'C': 8.79922543569107},{'C': 10.476157527896651},{'C': 6.97323567654597},{'C': 8.302175681319744},{'C': 12.472674708860454},{'C': 16.68100537200059},{'C': 18.73817422860383},{'C': 26.560877829466868}
split0_test_score,0.738706,0.737485,0.735043,0.738706,0.736264,0.733822,0.735043,0.735043,0.733822,0.735043,...,0.733822,0.733822,0.735043,0.736264,0.733822,0.733822,0.733822,0.732601,0.732601,0.728938
split1_test_score,0.746634,0.746634,0.746634,0.746634,0.744186,0.74541,0.744186,0.74541,0.746634,0.741738,...,0.724602,0.724602,0.728274,0.725826,0.725826,0.72705,0.724602,0.723378,0.724602,0.722154
split2_test_score,0.75765,0.75765,0.75765,0.752754,0.755202,0.755202,0.753978,0.755202,0.753978,0.752754,...,0.736842,0.735618,0.734394,0.734394,0.735618,0.734394,0.735618,0.73317,0.731946,0.730722
split3_test_score,0.758874,0.75765,0.756426,0.756426,0.753978,0.753978,0.750306,0.750306,0.752754,0.750306,...,0.73317,0.73317,0.729498,0.729498,0.730722,0.729498,0.728274,0.729498,0.729498,0.730722


In [113]:
# Predictions on test sample
pred = search.predict(get_word_vectors(test['description']))

In [114]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [115]:
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [116]:
submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

In [24]:
from sklearn.linear_model import SGDClassifier

# Review

To review this module: 
* Continue working on the Kaggle comeptition
* Find another text classification task to work on